# Multinomial Naive Bayes
다항분포 나이브 베이즈를 활용한 영화 리뷰 분류
영화 리뷰에 다항분포 나이브 베이즈를 활용하여 영화 리뷰가 긍정적인지 부정적인지 분류한다.

In [1]:
import pandas as pd
% config Completer.use_jedi = False
import warnings
warnings.filterwarnings(action='ignore')

In [2]:
# 다항분포 나이브 베이즈를 사용하기 위한 라이브러리를 import 한다.
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer
# 모델 정확도 평가를 위한 라이브러리를 import 한다.
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score

1. 데이터 획득
- movie_review: 영화 리뷰
- type: 긍정/부정

In [3]:
# 학습 데이터 준비
review_list = [
    {'movie_review': 'this is great great movie. I will watch again', 'type': 'positive'},
    {'movie_review': 'I like this movie', 'type': 'positive'},
    {'movie_review': 'amazing movie in this year', 'type': 'positive'},
    {'movie_review': 'cool my boyfriend also said the movie is cool', 'type': 'positive'},
    {'movie_review': 'awesome of the awesome movie ever', 'type': 'positive'},
    {'movie_review': 'shame I wasted money and time', 'type': 'negative'},
    {'movie_review': 'regret on this move. I will never never what movie from this director', 'type': 'negative'},
    {'movie_review': 'I do not like this movie', 'type': 'negative'},
    {'movie_review': 'I do not like actors in this movie', 'type': 'negative'},
    {'movie_review': 'boring boring sleeping movie', 'type': 'negative'}
]
df = pd.DataFrame(review_list)
df

,movie_review,type
0,this is great great movie. I will watch again,positive
1,I like this movie,positive
2,amazing movie in this year,positive
3,cool my boyfriend also said the movie is cool,positive
4,awesome of the awesome movie ever,positive
5,shame I wasted money and time,negative
6,regret on this move. I will never never what m...,negative
7,I do not like this movie,negative
8,I do not like actors in this movie,negative
9,boring boring sleeping movie,negative


2. 학습 데이터 다루기
* scikit-learn의 다항분포 나이브 베이즈 분류기는 <strong>숫자</strong>만 다루기 때문에 True와 False를 1과 0으로 치환한다.
영화 리뷰(movie_review)로 학습을 진행하고 레이블은 label을 사용해서 스팸 메일 여부를 판단한다.

In [4]:
df['label'] = df['type'].map({'positive': 1, 'negative': 0})
df

,movie_review,type,label
0,this is great great movie. I will watch again,positive,1
1,I like this movie,positive,1
2,amazing movie in this year,positive,1
3,cool my boyfriend also said the movie is cool,positive,1
4,awesome of the awesome movie ever,positive,1
5,shame I wasted money and time,negative,0
6,regret on this move. I will never never what m...,negative,0
7,I do not like this movie,negative,0
8,I do not like actors in this movie,negative,0
9,boring boring sleeping movie,negative,0


학습에 사용할 데이터와 레이블(클래스, 타겟, 답)로 분리한다.

In [6]:
df_x = df['movie_review']
df_x

0        this is great great movie. I will watch again
1                                    I like this movie
2                           amazing movie in this year
3        cool my boyfriend also said the movie is cool
4                    awesome of the awesome movie ever
5                        shame I wasted money and time
6    regret on this move. I will never never what m...
7                             I do not like this movie
8                   I do not like actors in this movie
9                         boring boring sleeping movie
Name: movie_review, dtype: object

In [7]:
df_y = df['label']
df_y

0    1
1    1
2    1
3    1
4    1
5    0
6    0
7    0
8    0
9    0
Name: label, dtype: int64

다항분포 나이브 베이즈의 입력 데이터는 고정된 크기의 벡터로써 각 인덱스는 단어의 출현 빈도수를 나타내야 한다.

In [8]:
# 베르누이 나이브 베이즈는 같은 단어가 여러 번 출현하더라도 무조건 1을 리턴해야 하기 때문에 binary=True 옵션을 지정하지만
# 다항분포 나이브 베이즈는 같은 단어가 여러 번 출현하면 출현한 빈도수를 나타내야 하므로 binary=False 옵션을 지정하지 않는다.
# cv = CountVectorizer(binary=True)   # 베르누이 나이브 베이즈
cv = CountVectorizer()  # 다항분포 나이브 베이즈
x_train = cv.fit_transform(df_x)
encoded_input = x_train.toarray()  # numpy 타입의 배열 데이터로 변환한다.
print(type(encoded_input))
encoded_input

<class 'numpy.ndarray'>


array([[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 1, 0, 0, 0, 1, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1],
       [0, 0, 1, 0, 0, 0, 0, 1, 2, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0,
        0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
        0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 2,
        0, 0, 1, 1, 0, 0, 0, 0, 2, 0, 0, 0, 1, 1, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0,
        1, 0, 0, 0, 0, 0, 0, 0

In [9]:
print(type(cv.get_feature_names_out()))
cv.get_feature_names_out()

<class 'numpy.ndarray'>


array(['actors', 'again', 'also', 'amazing', 'and', 'awesome', 'boring',
       'boyfriend', 'cool', 'director', 'do', 'ever', 'from', 'great',
       'in', 'is', 'like', 'money', 'move', 'movie', 'my', 'never', 'not',
       'of', 'on', 'regret', 'said', 'shame', 'sleeping', 'the', 'this',
       'time', 'wasted', 'watch', 'what', 'will', 'year'], dtype=object)

In [12]:
mnb = MultinomialNB()
y_train = df_y.astype('int')
mnb.fit(x_train, y_train)

MultinomialNB()

In [13]:
# 테스트 데이터 학습하기
test_feedback_list = [
    {'movie_review': 'great great great movie ever', 'type': 'positive'},
    {'movie_review': 'I like this amazing movie', 'type': 'positive'},
    {'movie_review': 'my boyfriend said great movie ever', 'type': 'positive'},
    {'movie_review': 'cool cool cool', 'type': 'positive'},
    {'movie_review': 'awesome boyfriend said cool movie ever', 'type': 'positive'},
    {'movie_review': 'shame shame shame', 'type': 'negative'},
    {'movie_review': 'awesome director shame movie boring movie', 'type': 'negative'},
    {'movie_review': 'do not like this movie', 'type': 'negative'},
    {'movie_review': 'I do not like this boring movie', 'type': 'negative'},
    {'movie_review': 'aweful terrible boring movie', 'type': 'negative'}
]
test_df = pd.DataFrame(test_feedback_list)
test_df['label'] = test_df['type'].map({'positive': 1, 'negative': 0})
test_df

,movie_review,type,label
0,great great great movie ever,positive,1
1,I like this amazing movie,positive,1
2,my boyfriend said great movie ever,positive,1
3,cool cool cool,positive,1
4,awesome boyfriend said cool movie ever,positive,1
5,shame shame shame,negative,0
6,awesome director shame movie boring movie,negative,0
7,do not like this movie,negative,0
8,I do not like this boring movie,negative,0
9,aweful terrible boring movie,negative,0


In [15]:
test_x = test_df['movie_review']
test_x

0                 great great great movie ever
1                    I like this amazing movie
2           my boyfriend said great movie ever
3                               cool cool cool
4       awesome boyfriend said cool movie ever
5                            shame shame shame
6    awesome director shame movie boring movie
7                       do not like this movie
8              I do not like this boring movie
9                 aweful terrible boring movie
Name: movie_review, dtype: object

In [16]:
test_y = test_df['label']
test_y

0    1
1    1
2    1
3    1
4    1
5    0
6    0
7    0
8    0
9    0
Name: label, dtype: int64

In [17]:
x_test = cv.transform(test_x)
y_test = test_y.astype(int)

In [18]:
predict = mnb.predict(x_test)
print(classification_report(y_test, predict))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00         5
           1       1.00      1.00      1.00         5

    accuracy                           1.00        10
   macro avg       1.00      1.00      1.00        10
weighted avg       1.00      1.00      1.00        10



In [19]:
print(f'정확도: {accuracy_score(y_test, predict):.2f}')

정확도: 1.00


In [20]:
comparison = pd.DataFrame({'실제값': y_test, '예측값': predict})
comparison

,실제값,예측값
0,1,1
1,1,1
2,1,1
3,1,1
4,1,1
5,0,0
6,0,0
7,0,0
8,0,0
9,0,0
